# **Fine-Tune Llama 2: Proyecto Quauhtlemallan**

## **Paso 1: Autenticación en Hugging Face**

In [1]:
!pip install -q datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
En

## **Paso 2: Carga y refactorización de dataset**

In [2]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('ChrisG19/Guatemala_info_2')

# Shuffle the dataset
dataset = dataset['train'].shuffle(seed=42)

# Split the dataset
train_dataset = dataset.select(range(2000))
test_dataset = dataset.select(range(2000, len(dataset)))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

# Apply the transformation
transformed_train_dataset = train_dataset.map(transform_conversation)
transformed_test_dataset = test_dataset.map(transform_conversation)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_dataset_v3.jsonl:   0%|          | 0.00/831k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4212 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2212 [00:00<?, ? examples/s]

## **Paso 3: Carga de datasets a HuggingFace**

In [3]:
transformed_train_dataset.push_to_hub("quauhtlemallan-llama2-train-2k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ChrisG19/quauhtlemallan-llama2-train-2k/commit/0bda89bbbbe425661e953d42bf984d36f8b82168', commit_message='Upload dataset', commit_description='', oid='0bda89bbbbe425661e953d42bf984d36f8b82168', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
transformed_test_dataset.push_to_hub("quauhtlemallan-llama2-test-2k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ChrisG19/quauhtlemallan-llama2-test-2k/commit/516ac1af7b7488bf787745fdac8450772141f094', commit_message='Upload dataset', commit_description='', oid='516ac1af7b7488bf787745fdac8450772141f094', pr_url=None, pr_revision=None, pr_num=None)